# 03 Models v2 - Forecasting + Interpretability

This notebook rebuilds the modeling workflow with three goals:
- reproducible ward/mesh baselines (Linear Regression, Random Forest, LightGBM)
- SHAP exports that Streamlit can read for global + local explainability
- a lightweight PyTorch LSTM baseline for wards, giving us a neural alternative without bloating runtime

Running everything end-to-end produces the CSV/pickle artifacts consumed by the dashboard.

In [1]:
# sets up imports, configuration, and helper utilities.
from __future__ import annotations

import json
import math
import warnings
from pathlib import Path
from typing import Dict, List

import joblib
import lightgbm as lgb
import numpy as np
import pandas as pd
import shap
import torch
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import DataLoader, Dataset

warnings.filterwarnings("ignore", category=UserWarning)

np.random.seed(42)
torch.manual_seed(42)

NOTEBOOK_DIR = Path.cwd()
DATA_DIR = NOTEBOOK_DIR
if not (DATA_DIR / "main_features.parquet").exists():
    candidate = NOTEBOOK_DIR / "test_notebooks"
    if (candidate / "main_features.parquet").exists():
        DATA_DIR = candidate

OUTPUT_DIR = DATA_DIR
SHAP_DIR = DATA_DIR / "shap_outputs"
for folder in [OUTPUT_DIR, SHAP_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

TRAIN_END = "2019-Q4"
VAL_END = "2021-Q4"

SEQ_LEN = 8
BATCH_SIZE = 128
MAX_EPOCHS = 75
LEARNING_RATE = 1e-3
EARLY_STOPPING_PATIENCE = 8
TARGET_SCALE = 1_000_000  # scale LSTM targets to keep gradients stable
TREE_MODELS = {"RandomForest", "LightGBM"}


def period_to_order(period_key: str) -> int:
    year, quarter = period_key.split("-Q")
    return int(year) * 4 + (int(quarter) - 1)


def period_to_float(period_key: str) -> float:
    return period_to_order(period_key) / 4.0


def evaluate_sets(y_true, y_pred) -> Dict[str, float]:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    return {"mae": mae, "rmse": float(np.sqrt(mse)), "r2": r2_score(y_true, y_pred)}


def temporal_split(df: pd.DataFrame, train_end: str, val_end: str):
    order_series = df["PeriodKey"].apply(period_to_order)
    train_mask = order_series <= period_to_order(train_end)
    val_mask = (order_series > period_to_order(train_end)) & (order_series <= period_to_order(val_end))
    test_mask = order_series > period_to_order(val_end)
    return df[train_mask], df[val_mask], df[test_mask]


def add_temporal_features(df: pd.DataFrame, group_col: str, target_col: str) -> pd.DataFrame:
    df = df.sort_values([group_col, "PeriodKey"]).copy()
    df["PeriodNum"] = df["PeriodKey"].astype(str).apply(period_to_float)
    for lag in [1, 4]:
        df[f"{target_col}_lag{lag}"] = df.groupby(group_col)[target_col].shift(lag)
    df[f"{target_col}_growth_qoq"] = (df[target_col] / df[f"{target_col}_lag1"] - 1) * 100
    df[f"{target_col}_growth_yoy"] = (df[target_col] / df[f"{target_col}_lag4"] - 1) * 100
    df[f"{target_col}_ma4q"] = (
        df.groupby(group_col)[target_col]
        .rolling(window=4, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )
    df[f"{target_col}_std4q"] = (
        df.groupby(group_col)[target_col]
        .rolling(window=4, min_periods=1)
        .std()
        .reset_index(level=0, drop=True)
    )
    return df


MODEL_FACTORIES = {
    "LinearRegression": lambda: LinearRegression(),
    "RandomForest": lambda: RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1),
    "LightGBM": lambda: lgb.LGBMRegressor(
        n_estimators=600,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
    ),
}

TRAIN_END_ORDER = period_to_order(TRAIN_END)
VAL_END_ORDER = period_to_order(VAL_END)

Using device: cuda


In [2]:
# loads cleaned feature tables and hedonic references used downstream.
MAIN_FEATURES_PATH = DATA_DIR / "main_features.parquet"
MAIN_FEATURES_CSV = DATA_DIR / "main_features.csv"
MESH_FEATURES_PATH = DATA_DIR / "mesh_quarter_features.csv"

if not MAIN_FEATURES_PATH.exists():
    raise FileNotFoundError(MAIN_FEATURES_PATH)
if not MESH_FEATURES_PATH.exists():
    raise FileNotFoundError(MESH_FEATURES_PATH)

main_df = pd.read_parquet(MAIN_FEATURES_PATH)
if MAIN_FEATURES_CSV.exists():
    print("Loaded main_features.csv for sharing; modeling uses the Parquet table.")
mesh_panel_raw = pd.read_csv(MESH_FEATURES_PATH)

main_df["Mesh250m"] = main_df["Mesh250m"].astype(str)
main_df.loc[main_df["Mesh250m"].str.lower() == "nan", "Mesh250m"] = np.nan
main_df["WardName"] = main_df["Municipality_en"].fillna(main_df["Municipality"]).fillna("Unknown")

mesh_panel_raw["Mesh250m"] = mesh_panel_raw["Mesh250m"].astype(str)
mesh_panel_raw.loc[mesh_panel_raw["Mesh250m"].str.lower() == "nan", "Mesh250m"] = np.nan

hedonic_paths = {
    "overall": DATA_DIR / "hedonic_index_overall.csv",
    "ward_full": DATA_DIR / "hedonic_index_by_ward.csv",
    "mesh_full": DATA_DIR / "hedonic_index_by_mesh.csv",
    "ward_train": DATA_DIR / "hedonic_index_by_ward_trainmodel.csv",
    "mesh_train": DATA_DIR / "hedonic_index_by_mesh_trainmodel.csv",
}


def _load_optional_csv(path: Path) -> pd.DataFrame:
    if path.exists():
        df = pd.read_csv(path)
        if "PeriodKey" in df.columns:
            df["PeriodKey"] = df["PeriodKey"].astype(str)
        return df
    return pd.DataFrame()


hedonic_index_overall = _load_optional_csv(hedonic_paths["overall"])
ward_hedonic_index_full = _load_optional_csv(hedonic_paths["ward_full"])
mesh_hedonic_index_full = _load_optional_csv(hedonic_paths["mesh_full"])
ward_hedonic_index = _load_optional_csv(hedonic_paths["ward_train"])
mesh_hedonic_index = _load_optional_csv(hedonic_paths["mesh_train"])

HEDONIC_AVAILABLE = not hedonic_index_overall.empty

print(f"Transactions loaded: {len(main_df):,}")
print(f"Mesh quarters loaded: {len(mesh_panel_raw):,}")
print(f"Time span: {main_df['PeriodKey'].min()} -> {main_df['PeriodKey'].max()}")
if not HEDONIC_AVAILABLE:
    print("Hedonic index CSVs not found; skipping hedonic comparison plots.")

Loaded main_features.csv for sharing; modeling uses the Parquet table.
Transactions loaded: 485,093
Mesh quarters loaded: 26,842
Time span: 2005-Q3 -> 2025-Q1
Hedonic index CSVs not found; skipping hedonic comparison plots.


## Ward-level aggregation

In [3]:
# aggregates ward-level metrics and engineers time features.
ward_panel = main_df.groupby(["WardName", "PeriodKey"], dropna=False).agg({
    "PricePerSqM": ["median", "mean", "std"],
    "TradePriceValue": "median",
    "BuildingAge": "mean",
    "AreaSqM": "mean",
    "Mesh250m": "nunique",
}).reset_index()
ward_panel.columns = [
    "Ward", "PeriodKey", "MedianPriceSqM", "MeanPriceSqM", "StdPriceSqM",
    "MedianTotalPrice", "AvgBuildingAge", "AvgArea", "ActiveMeshes"
]
ward_counts = (
    main_df.groupby(["WardName", "PeriodKey"], dropna=False)
    .size()
    .reset_index(name="TransactionCount")
    .rename(columns={"WardName": "Ward"})
)
ward_panel = ward_panel.merge(ward_counts, on=["Ward", "PeriodKey"], how="left")
ward_coordinates = (
    main_df.dropna(subset=["Latitude", "Longitude"])
    .groupby("WardName")
    .agg(WardLat=("Latitude", "median"), WardLon=("Longitude", "median"))
    .reset_index()
    .rename(columns={"WardName": "Ward"})
)
ward_panel = ward_panel.merge(ward_coordinates, on="Ward", how="left")

if not ward_hedonic_index_full.empty and {"Ward", "PeriodKey", "WardHedonicIndexFull"}.issubset(ward_hedonic_index_full.columns):
    ward_panel = ward_panel.merge(
        ward_hedonic_index_full[["Ward", "PeriodKey", "WardHedonicIndexFull"]],
        on=["Ward", "PeriodKey"],
        how="left",
    )
else:
    ward_panel["WardHedonicIndexFull"] = np.nan

if not ward_hedonic_index.empty and {"Ward", "PeriodKey", "WardHedonicIndex"}.issubset(ward_hedonic_index.columns):
    ward_panel = ward_panel.merge(
        ward_hedonic_index[["Ward", "PeriodKey", "WardHedonicIndex"]],
        on=["Ward", "PeriodKey"],
        how="left",
    )
else:
    if "WardHedonicIndex" not in ward_panel.columns:
        ward_panel["WardHedonicIndex"] = np.nan

ward_panel = add_temporal_features(ward_panel, "Ward", "MedianPriceSqM")
ward_panel["Quarter"] = ward_panel["PeriodKey"].str.extract(r"Q(\d)")[0].astype(int)
ward_panel = pd.get_dummies(ward_panel, columns=["Quarter"], prefix="Q", drop_first=True)
ward_panel["TimeTrend"] = ward_panel.groupby("Ward").cumcount()
ward_encoder = LabelEncoder()
ward_panel["Ward_encoded"] = ward_encoder.fit_transform(ward_panel["Ward"].fillna("Unknown"))
ward_panel["Order"] = ward_panel["PeriodKey"].apply(period_to_order)

print(f"Ward panel shape: {ward_panel.shape}")
ward_panel.head()

Ward panel shape: (2207, 27)


,Ward,PeriodKey,MedianPriceSqM,MeanPriceSqM,StdPriceSqM,MedianTotalPrice,AvgBuildingAge,AvgArea,ActiveMeshes,TransactionCount,...,MedianPriceSqM_growth_qoq,MedianPriceSqM_growth_yoy,MedianPriceSqM_ma4q,MedianPriceSqM_std4q,Q_2,Q_3,Q_4,TimeTrend,Ward_encoded,Order
0,Adachi Ward,2005-Q3,272077.922078,317513.105806,211710.128325,28500000.0,12.370166,172.445255,20,274,...,NaN,NaN,272077.922078,NaN,False,True,False,0,0,8022
1,Adachi Ward,2005-Q4,302000.000000,331062.433878,148225.544631,32000000.0,10.218750,152.871257,18,334,...,10.997613,NaN,287038.961039,21158.104206,False,False,True,1,0,8023
2,Adachi Ward,2006-Q1,296666.666667,330761.524037,179748.518250,28000000.0,12.205000,211.641304,18,276,...,-1.766004,NaN,290248.196248,15960.271260,False,False,False,2,0,8024
3,Adachi Ward,2006-Q2,316025.641026,338663.621605,134662.595791,30000000.0,10.297561,195.388489,17,278,...,6.525497,NaN,296692.557443,18328.647993,True,False,False,3,0,8025
4,Adachi Ward,2006-Q3,343167.701863,381259.114252,369920.460052,31000000.0,10.588745,152.731544,17,298,...,8.588563,26.128463,314465.002389,20804.146935,False,True,False,4,0,8026


## Mesh-level aggregation

In [4]:
# builds mesh-level panel with ward context features.
mesh_panel = mesh_panel_raw.copy()
mesh_panel["PeriodKey"] = mesh_panel["PeriodKey"].astype(str)
mesh_panel = add_temporal_features(mesh_panel, "Mesh250m", "mesh_median_ppsqm")
mesh_panel["Quarter"] = mesh_panel["PeriodKey"].str.extract(r"Q(\d)")[0].astype(int)
mesh_panel = pd.get_dummies(mesh_panel, columns=["Quarter"], prefix="Q", drop_first=True)
mesh_panel["TimeTrend"] = mesh_panel.groupby("Mesh250m").cumcount()

mesh_to_ward = (
    main_df[["Mesh250m", "WardName"]]
    .dropna(subset=["Mesh250m"])
    .drop_duplicates()
    .rename(columns={"WardName": "Ward"})
)
mesh_panel = mesh_panel.merge(mesh_to_ward, on="Mesh250m", how="left")
mesh_panel = mesh_panel.merge(
    ward_panel[["Ward", "PeriodKey", "WardHedonicIndex", "WardHedonicIndexFull", "WardLat", "WardLon"]],
    on=["Ward", "PeriodKey"],
    how="left",
)
mesh_coordinates = (
    main_df.dropna(subset=["Mesh250m", "Latitude", "Longitude"])
    .groupby("Mesh250m")
    .agg(MeshLat=("Latitude", "median"), MeshLon=("Longitude", "median"))
    .reset_index()
)
mesh_panel = mesh_panel.merge(mesh_coordinates, on="Mesh250m", how="left")

if not mesh_hedonic_index_full.empty and {"Mesh250m", "PeriodKey", "MeshHedonicIndexFull"}.issubset(mesh_hedonic_index_full.columns):
    mesh_panel = mesh_panel.merge(
        mesh_hedonic_index_full[["Mesh250m", "PeriodKey", "MeshHedonicIndexFull"]],
        on=["Mesh250m", "PeriodKey"],
        how="left",
    )
else:
    mesh_panel["MeshHedonicIndexFull"] = mesh_panel.get("MeshHedonicIndexFull", np.nan)

if not mesh_hedonic_index.empty and {"Mesh250m", "PeriodKey", "MeshHedonicIndex"}.issubset(mesh_hedonic_index.columns):
    mesh_panel = mesh_panel.merge(
        mesh_hedonic_index[["Mesh250m", "PeriodKey", "MeshHedonicIndex"]],
        on=["Mesh250m", "PeriodKey"],
        how="left",
    )
else:
    if "MeshHedonicIndex" not in mesh_panel.columns:
        mesh_panel["MeshHedonicIndex"] = np.nan

mesh_panel["Ward_encoded"] = ward_encoder.transform(mesh_panel["Ward"].fillna("Unknown"))
mesh_panel["Order"] = mesh_panel["PeriodKey"].apply(period_to_order)

print(f"Mesh panel shape: {mesh_panel.shape}")
mesh_panel.head()

Mesh panel shape: (41072, 31)


,Mesh250m,PeriodKey,mesh_transaction_count,mesh_median_ppsqm,mesh_mean_ppsqm,mesh_price_std,mesh_price_iqr,mesh_avg_age,mesh_avg_area,PeriodNum,...,WardHedonicIndex,WardHedonicIndexFull,WardLat,WardLon,MeshLat,MeshLon,MeshHedonicIndexFull,MeshHedonicIndex,Ward_encoded,Order
0,53392546,2005-Q3,3,446153.846154,543728.283085,294852.624070,282484.498450,3.000000,3368.000000,2005.50,...,NaN,NaN,35.572571,139.708489,35.540512,139.707448,NaN,NaN,17,8022
1,53392546,2005-Q4,1,550000.000000,550000.000000,NaN,0.000000,2.000000,60.000000,2005.75,...,NaN,NaN,35.572571,139.708489,35.540512,139.707448,NaN,NaN,17,8023
2,53392546,2006-Q3,11,950000.000000,815609.239246,302013.869517,480000.000000,3.800000,39.545455,2006.50,...,NaN,NaN,35.572571,139.708489,35.540512,139.707448,NaN,NaN,17,8026
3,53392546,2007-Q1,6,557017.543860,576573.280521,243349.016575,365744.891074,10.500000,190.833333,2007.00,...,NaN,NaN,35.572571,139.708489,35.540512,139.707448,NaN,NaN,17,8028
4,53392546,2007-Q3,3,600000.000000,627777.777778,259450.987307,258333.333333,8.666667,48.333333,2007.50,...,NaN,NaN,35.572571,139.708489,35.540512,139.707448,NaN,NaN,17,8030


## Train classical baselines

In [5]:
# trains linear, forest, and lgbm baselines for wards and meshes.
WARD_TARGET = "MedianPriceSqM"
WARD_FEATURE_COLS = [
    "MedianPriceSqM_lag1", "MedianPriceSqM_lag4", "MedianPriceSqM_growth_qoq",
    "MedianPriceSqM_growth_yoy", "MedianPriceSqM_ma4q", "MedianPriceSqM_std4q",
    "TransactionCount", "AvgBuildingAge", "AvgArea", "ActiveMeshes", "TimeTrend",
    "Ward_encoded", "Q_2", "Q_3", "Q_4"
]

MESH_TARGET = "mesh_median_ppsqm"
MESH_FEATURE_COLS = [
    "mesh_median_ppsqm_lag1", "mesh_median_ppsqm_lag4", "mesh_median_ppsqm_growth_qoq",
    "mesh_median_ppsqm_growth_yoy", "mesh_median_ppsqm_ma4q", "mesh_median_ppsqm_std4q",
    "mesh_transaction_count", "mesh_avg_age", "mesh_avg_area", "TimeTrend",
    "Ward_encoded", "WardHedonicIndex", "MeshHedonicIndex"
]


def prepare_model_splits(panel: pd.DataFrame, feature_cols: List[str], target_col: str):
    feature_cols = [c for c in feature_cols if c in panel.columns]
    train_df, val_df, test_df = temporal_split(panel.dropna(subset=[target_col]), TRAIN_END, VAL_END)
    splits = {
        "train": train_df.dropna(subset=feature_cols + [target_col]).copy(),
        "val": val_df.dropna(subset=feature_cols + [target_col]).copy(),
        "test": test_df.dropna(subset=feature_cols + [target_col]).copy(),
    }
    return splits, feature_cols


def train_regression_models(panel: pd.DataFrame, level_name: str, target_col: str, feature_cols: List[str]):
    splits, feature_cols = prepare_model_splits(panel, feature_cols, target_col)
    results, prediction_frames, trained_models = [], [], {}

    for name, factory in MODEL_FACTORIES.items():
        train_df = splits["train"]
        if train_df.empty:
            continue
        model = factory()
        model.fit(train_df[feature_cols], train_df[target_col])
        preds = {split: model.predict(split_df[feature_cols]) for split, split_df in splits.items()}
        metrics = {
            split: evaluate_sets(split_df[target_col], preds[split])
            for split, split_df in splits.items() if not split_df.empty
        }
        results.append({
            "Model": name,
            **{f"{split}_{metric}": values[metric] for split, values in metrics.items() for metric in ["mae", "rmse", "r2"]},
        })
        for split, split_df in splits.items():
            if split_df.empty:
                continue
            payload = {
                "Level": level_name,
                "Model": name,
                "Split": split,
                "PeriodKey": split_df["PeriodKey"].values,
                "Actual": split_df[target_col].values,
                "Predicted": preds[split],
            }
            for col in ["Ward", "WardLat", "WardLon", "Mesh250m", "MeshLat", "MeshLon"]:
                payload[col] = split_df.get(col, pd.Series(np.nan, index=split_df.index)).values
            prediction_frames.append(pd.DataFrame(payload))
        model_path = OUTPUT_DIR / f"{level_name.lower()}_model_{name.lower()}.pkl"
        joblib.dump(model, model_path)
        trained_models[name] = model

    results_df = pd.DataFrame(results)
    results_df["Level"] = level_name
    predictions_df = pd.concat(prediction_frames, ignore_index=True) if prediction_frames else pd.DataFrame()
    return results_df, predictions_df, trained_models, splits, feature_cols


ward_results_df, ward_predictions_df, ward_models, ward_splits, ward_features = train_regression_models(
    ward_panel, "Ward", WARD_TARGET, WARD_FEATURE_COLS
)
mesh_results_df, mesh_predictions_df, mesh_models, mesh_splits, mesh_features = train_regression_models(
    mesh_panel, "Mesh", MESH_TARGET, MESH_FEATURE_COLS
)

ward_results_df.sort_values("test_mae")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2413
[LightGBM] [Info] Number of data points in the train set: 1507, number of used features: 15
[LightGBM] [Info] Start training from score 579864.310141


,Model,train_mae,train_rmse,train_r2,val_mae,val_rmse,val_r2,test_mae,test_rmse,test_r2,Level
0,LinearRegression,15289.386684,21997.566438,0.994241,14414.898507,19777.222355,0.997015,18594.707647,29651.119045,0.995358,Ward
1,RandomForest,3940.526974,6097.334658,0.999558,14886.734120,26010.622263,0.994837,44302.315954,112688.612153,0.932945,Ward
2,LightGBM,1117.026318,1858.730250,0.999959,15949.871880,29862.142839,0.993195,48038.703388,116652.766178,0.928145,Ward


## SHAP outputs for Streamlit

In [6]:
# exports shap summaries plus local explanations for tree models.
def save_tree_shap_outputs(level_name: str, model_name: str, model, split_df: pd.DataFrame, feature_cols: List[str]):
    if split_df.empty:
        print(f"No data for SHAP: {level_name} {model_name}")
        return
    try:
        explainer = shap.TreeExplainer(model)
    except Exception as exc:
        print(f"SHAP explainer failed for {level_name} {model_name}: {exc}")
        return

    sample = split_df.sample(n=min(600, len(split_df)), random_state=42)
    shap_values = explainer.shap_values(sample[feature_cols])
    if isinstance(shap_values, list):
        shap_values = shap_values[0]
    shap_abs = np.abs(shap_values)
    summary_df = pd.DataFrame({
        "Feature": feature_cols,
        "MeanAbsSHAP": shap_abs.mean(axis=0),
    }).sort_values("MeanAbsSHAP", ascending=False)
    summary_path = SHAP_DIR / f"{level_name.lower()}_{model_name.lower()}_shap_summary.csv"
    summary_df.to_csv(summary_path, index=False)

    meta_cols = ["Ward", "Mesh250m", "PeriodKey"]
    local_records = []
    sample_reset = sample.reset_index(drop=True)
    for i in range(len(sample_reset)):
        meta = {col: sample_reset.iloc[i].get(col) for col in meta_cols if col in sample_reset.columns}
        base = {
            "ObservationIndex": int(i),
            "Model": model_name,
            "Level": level_name,
            **meta,
        }
        for feature_idx, feature in enumerate(feature_cols):
            local_records.append({
                **base,
                "Feature": feature,
                "FeatureValue": float(sample_reset.iloc[i][feature]),
                "SHAPValue": float(shap_values[i, feature_idx]),
            })
    local_df = pd.DataFrame(local_records)
    local_path = SHAP_DIR / f"{level_name.lower()}_{model_name.lower()}_shap_local.csv"
    local_df.to_csv(local_path, index=False)

    try:
        plt.figure(figsize=(8, 4))
        shap.summary_plot(shap_values, sample[feature_cols], plot_type="bar", show=False)
        plt.tight_layout()
        plt.savefig(SHAP_DIR / f"{level_name.lower()}_{model_name.lower()}_shap_bar.png", dpi=200)
        plt.close()
    except Exception as exc:
        print(f"SHAP bar plot skipped for {level_name} {model_name}: {exc}")

    metadata_path = SHAP_DIR / f"{level_name.lower()}_{model_name.lower()}_shap_metadata.json"
    expected_value = explainer.expected_value
    if isinstance(expected_value, list):
        expected_value = expected_value[0]
    with metadata_path.open("w", encoding="utf-8") as fh:
        json.dump({
            "expected_value": float(expected_value),
            "feature_cols": feature_cols,
            "n_samples": len(sample_reset),
        }, fh, indent=2)


for level_name, models, splits, features in [
    ("Ward", ward_models, ward_splits, ward_features),
    ("Mesh", mesh_models, mesh_splits, mesh_features),
]:
    for model_name, model in models.items():
        if model_name not in TREE_MODELS:
            continue
        save_tree_shap_outputs(level_name, model_name, model, splits["val"], features)

C:\Users\ignit\AppData\Local\Temp\ipykernel_4600\3032209221.py:48: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_values, sample[feature_cols], plot_type="bar", show=False)
C:\Users\ignit\AppData\Local\Temp\ipykernel_4600\3032209221.py:61: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "expected_value": float(expected_value),
C:\Users\ignit\AppData\Local\Temp\ipykernel_4600\3032209221.py:48: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  s

## PyTorch LSTM for ward forecasts

In [7]:
# defines sequence helpers and trains the torch lstm.

class SequenceDataset(Dataset):
    def __init__(self, features: np.ndarray, targets: np.ndarray):
        self.x = torch.tensor(features, dtype=torch.float32)
        self.y = torch.tensor(targets, dtype=torch.float32).unsqueeze(-1)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


class PriceLSTM(nn.Module):
    def __init__(self, input_dim: int, hidden_size: int = 96, dropout: float = 0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        output, _ = self.lstm(x)
        last_hidden = output[:, -1, :]
        last_hidden = self.dropout(last_hidden)
        return self.fc(last_hidden)


def prepare_sequence_data(panel: pd.DataFrame, feature_cols: List[str], target_col: str, group_col: str = "Ward"):
    feature_cols = [c for c in feature_cols if c in panel.columns]
    df = panel.dropna(subset=feature_cols + [target_col]).copy()
    df["Order"] = df["PeriodKey"].apply(period_to_order)
    train_mask = df["Order"] <= TRAIN_END_ORDER
    means = df.loc[train_mask, feature_cols].mean()
    stds = df.loc[train_mask, feature_cols].std().replace(0, 1)
    target_raw = df[target_col].astype(np.float32).copy()
    df[feature_cols] = (df[feature_cols] - means) / stds
    df["_target_raw"] = target_raw

    sequence_data = {
        "train": {"features": [], "targets": [], "meta": []},
        "val": {"features": [], "targets": [], "meta": []},
        "test": {"features": [], "targets": [], "meta": []},
    }

    for key, group in df.groupby(group_col):
        group = group.sort_values("Order")
        if len(group) <= SEQ_LEN:
            continue
        feats = group[feature_cols].values.astype(np.float32)
        targets = group["_target_raw"].values.astype(np.float32)
        for idx in range(SEQ_LEN, len(group)):
            window = feats[idx - SEQ_LEN: idx]
            y = targets[idx]
            order_val = group["Order"].iloc[idx]
            if order_val <= TRAIN_END_ORDER:
                split = "train"
            elif order_val <= VAL_END_ORDER:
                split = "val"
            else:
                split = "test"
            meta = {
                "Ward": key,
                "PeriodKey": group["PeriodKey"].iloc[idx],
                "WardLat": group["WardLat"].iloc[idx] if "WardLat" in group.columns else np.nan,
                "WardLon": group["WardLon"].iloc[idx] if "WardLon" in group.columns else np.nan,
            }
            sequence_data[split]["features"].append(window)
            sequence_data[split]["targets"].append(y)
            sequence_data[split]["meta"].append(meta)

    for split in sequence_data:
        if sequence_data[split]["features"]:
            sequence_data[split]["features"] = np.stack(sequence_data[split]["features"])
            sequence_data[split]["targets"] = np.array(sequence_data[split]["targets"], dtype=np.float32)
            sequence_data[split]["meta"] = pd.DataFrame(sequence_data[split]["meta"])
        else:
            sequence_data[split]["features"] = np.empty((0, SEQ_LEN, len(feature_cols)), dtype=np.float32)
            sequence_data[split]["targets"] = np.empty((0,), dtype=np.float32)
            sequence_data[split]["meta"] = pd.DataFrame(columns=["Ward", "PeriodKey", "WardLat", "WardLon"])
    scaler_info = {"mean": means.to_dict(), "std": stds.to_dict(), "features": feature_cols}
    return sequence_data, scaler_info


def _build_loader(split_data):
    if split_data["features"].size == 0:
        return None
    targets = split_data["targets"] / TARGET_SCALE
    dataset = SequenceDataset(split_data["features"], targets)
    return DataLoader(dataset, batch_size=min(BATCH_SIZE, len(dataset)), shuffle=True)


def _evaluate_loader(model, loader):
    if loader is None:
        return np.nan, np.nan, np.nan
    preds, trues = [], []
    model.eval()
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            outputs = model(xb)
            preds.append(outputs.cpu().numpy())
            trues.append(yb.cpu().numpy())
    preds = np.vstack(preds).flatten() * TARGET_SCALE
    trues = np.vstack(trues).flatten() * TARGET_SCALE
    metrics = evaluate_sets(trues, preds)
    return metrics["mae"], metrics["rmse"], metrics["r2"]


def train_lstm(sequence_data, input_dim: int):
    train_loader = _build_loader(sequence_data["train"])
    val_loader = _build_loader(sequence_data["val"])
    model = PriceLSTM(input_dim=input_dim).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.L1Loss()
    best_state = None
    best_val = math.inf
    patience_ctr = 0

    for _ in range(MAX_EPOCHS):
        if train_loader is None:
            break
        model.train()
        for xb, yb in train_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
        val_mae, _, _ = _evaluate_loader(model, val_loader)
        if val_mae < best_val:
            best_val = val_mae
            patience_ctr = 0
            best_state = model.state_dict()
        else:
            patience_ctr += 1
            if patience_ctr >= EARLY_STOPPING_PATIENCE:
                break
    if best_state is not None:
        model.load_state_dict(best_state)
    return model


def predict_sequences(model, split_data):
    if split_data["features"].size == 0:
        return np.array([])
    dummy_targets = np.zeros_like(split_data["targets"], dtype=np.float32)
    loader = DataLoader(SequenceDataset(split_data["features"], dummy_targets), batch_size=BATCH_SIZE, shuffle=False)
    preds = []
    model.eval()
    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(DEVICE)
            outputs = model(xb)
            preds.append(outputs.cpu().numpy())
    preds = np.vstack(preds).flatten() * TARGET_SCALE
    return preds


WARD_LSTM_FEATURES = [
    "MedianPriceSqM", "MedianPriceSqM_lag1", "MedianPriceSqM_ma4q",
    "MedianPriceSqM_growth_qoq", "TransactionCount", "AvgBuildingAge", "ActiveMeshes"
]

sequence_data, scaler_info = prepare_sequence_data(ward_panel, WARD_LSTM_FEATURES, WARD_TARGET)
if sequence_data["train"]["features"].size:
    lstm_model = train_lstm(sequence_data, input_dim=sequence_data["train"]["features"].shape[-1])
    lstm_predictions = {}
    lstm_results = {}
    for split in ["train", "val", "test"]:
        preds = predict_sequences(lstm_model, sequence_data[split])
        lstm_predictions[split] = preds
        if preds.size:
            metrics = evaluate_sets(sequence_data[split]["targets"], preds)
            lstm_results[split] = metrics
    ward_lstm_results_df = (
        pd.DataFrame({
            "Model": ["TorchLSTM"],
            **{f"{split}_{metric}": [values.get(metric, np.nan)] for split, values in lstm_results.items() for metric in ["mae", "rmse", "r2"]},
        })
    )
    ward_lstm_results_df["Level"] = "Ward"
    lstm_prediction_frames = []
    for split in ["train", "val", "test"]:
        meta = sequence_data[split]["meta"].copy()
        if meta.empty:
            continue
        meta["Model"] = "TorchLSTM"
        meta["Level"] = "Ward"
        meta["Split"] = split
        meta["Actual"] = sequence_data[split]["targets"]
        meta["Predicted"] = lstm_predictions[split]
        lstm_prediction_frames.append(meta)
    ward_lstm_predictions_df = pd.concat(lstm_prediction_frames, ignore_index=True)
    torch.save(lstm_model.state_dict(), OUTPUT_DIR / "ward_model_torchlstm.pt")
    with (OUTPUT_DIR / "ward_model_torchlstm_features.json").open("w", encoding="utf-8") as fh:
        json.dump({"feature_cols": scaler_info["features"], **scaler_info}, fh, indent=2)
else:
    ward_lstm_results_df = pd.DataFrame(columns=["Model", "train_mae", "val_mae", "test_mae"]).assign(Level="Ward")
    ward_lstm_predictions_df = pd.DataFrame()
    lstm_model = None

ward_lstm_results_df


,Model,train_mae,train_rmse,train_r2,val_mae,val_rmse,val_r2,test_mae,test_rmse,test_r2,Level
0,TorchLSTM,26357.412866,37359.440062,0.983402,27351.768309,38194.829837,0.988868,38419.233201,68314.440846,0.975357,Ward


## Export artifacts for Streamlit

In [8]:
# writes csv artifacts for the streamlit dashboard.
all_results = pd.concat(
    [
        ward_results_df,
        mesh_results_df,
        ward_lstm_results_df,
    ],
    ignore_index=True,
    sort=False,
)
all_results.to_csv(OUTPUT_DIR / "model_results.csv", index=False)

ward_predictions_full = pd.concat(
    [
        ward_predictions_df,
        ward_lstm_predictions_df,
    ],
    ignore_index=True,
    sort=False,
)
ward_predictions_full.to_csv(OUTPUT_DIR / "ward_predictions_detailed.csv", index=False)

mesh_predictions_df.to_csv(OUTPUT_DIR / "mesh_predictions_detailed.csv", index=False)

viz_frames = []
if not ward_predictions_full.empty:
    ward_viz = ward_predictions_full.copy()
    ward_viz["Latitude"] = ward_viz["WardLat"]
    ward_viz["Longitude"] = ward_viz["WardLon"]
    viz_frames.append(ward_viz)
if not mesh_predictions_df.empty:
    mesh_viz = mesh_predictions_df.copy()
    mesh_viz["Latitude"] = mesh_viz["MeshLat"].fillna(mesh_viz["WardLat"])
    mesh_viz["Longitude"] = mesh_viz["MeshLon"].fillna(mesh_viz["WardLon"])
    viz_frames.append(mesh_viz)

if viz_frames:
    model_predictions_viz = pd.concat(viz_frames, ignore_index=True, sort=False)
    model_predictions_viz.to_csv(OUTPUT_DIR / "model_predictions_viz.csv", index=False)

print("Artifacts written:")
for path in [
    OUTPUT_DIR / "model_results.csv",
    OUTPUT_DIR / "ward_predictions_detailed.csv",
    OUTPUT_DIR / "mesh_predictions_detailed.csv",
    OUTPUT_DIR / "model_predictions_viz.csv",
]:
    if path.exists():
        print(f"- {path}")
print(f"SHAP files stored under {SHAP_DIR}")

Artifacts written:
- c:\Users\ignit\OneDrive\Desktop\Study\GeorgiaTech\CSE6242 - Fall 2025\Project\test_notebooks\model_results.csv
- c:\Users\ignit\OneDrive\Desktop\Study\GeorgiaTech\CSE6242 - Fall 2025\Project\test_notebooks\ward_predictions_detailed.csv
- c:\Users\ignit\OneDrive\Desktop\Study\GeorgiaTech\CSE6242 - Fall 2025\Project\test_notebooks\mesh_predictions_detailed.csv
- c:\Users\ignit\OneDrive\Desktop\Study\GeorgiaTech\CSE6242 - Fall 2025\Project\test_notebooks\model_predictions_viz.csv
SHAP files stored under c:\Users\ignit\OneDrive\Desktop\Study\GeorgiaTech\CSE6242 - Fall 2025\Project\test_notebooks\shap_outputs


### How to run
1. Execute every cell sequentially after refreshing `main_features` / `mesh_quarter_features`.
2. Point the Streamlit app to the CSVs in `test_notebooks/` and the SHAP files under `test_notebooks/shap_outputs/`.
3. (Optional) load `ward_model_torchlstm.pt` plus the JSON feature config if you want to reuse the LSTM weights elsewhere.